In [1]:
import os
import geopandas as gpd
from simpledbf import Dbf5     # use this because it's much faster compared to the gpd function

PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.


In [2]:
input_path = '../000-download_dbf/'

file_list = os.listdir(input_path)

In [3]:
# IMPORT ALL THE DATABASES INTO PANDAS DATAFRAMES

df_dict = {}

for file_name in file_list:

    if file_name[-4:] == '.dbf':

        dbf_path = input_path + '/' + file_name

        dbf = Dbf5(dbf_path)

        df = dbf.to_dataframe()

        df_dict[file_name] = df

        print('Importing DataFrame: ', file_name[:-4], ' of shape ', df.shape)

Import DataFrame:  gis_osm_buildings_a_free_1  of shape  (4576030, 5)
Import DataFrame:  gis_osm_landuse_a_free_1  of shape  (529284, 4)
Import DataFrame:  gis_osm_natural_a_free_1  of shape  (1198, 4)
Import DataFrame:  gis_osm_natural_free_1  of shape  (323206, 4)
Import DataFrame:  gis_osm_places_a_free_1  of shape  (672, 5)
Import DataFrame:  gis_osm_places_free_1  of shape  (34338, 5)
Import DataFrame:  gis_osm_pofw_a_free_1  of shape  (11809, 4)
Import DataFrame:  gis_osm_pofw_free_1  of shape  (1276, 4)
Import DataFrame:  gis_osm_pois_a_free_1  of shape  (92330, 4)
Import DataFrame:  gis_osm_pois_free_1  of shape  (191061, 4)
Import DataFrame:  gis_osm_railways_free_1  of shape  (16924, 7)
Import DataFrame:  gis_osm_roads_free_1  of shape  (1260180, 10)
Import DataFrame:  gis_osm_traffic_a_free_1  of shape  (56236, 4)
Import DataFrame:  gis_osm_traffic_free_1  of shape  (141234, 4)
Import DataFrame:  gis_osm_transport_a_free_1  of shape  (910, 4)
Import DataFrame:  gis_osm_trans

In [5]:
# ADD THE COORDINATES TO THE DATAFRAMES USING CORRESPONDING SHP FILES

for df_name, df in df_dict.items():

    print('Starting finding coordinates for ', df_name[:-4], ' of shape ', df.shape)

    shp_path = input_path + '/' + df_name[:-4] + '.shp'

    shp_series = gpd.read_file(shp_path)

    project_shp_series = shp_series.to_crs("epsg:4087")

    centroids = project_shp_series.centroid

    df['lon'] = centroids.geometry.x
    df['lat'] = centroids.geometry.y

    print('Added coordinates to ', df_name[:-4], ', now shape is ', df.shape )

Starting finding coordinates for  gis_osm_buildings_a_free_1  of shape  (4576030, 5)
Added coordinates to  gis_osm_buildings_a_free_1 , now shape is  (4576030, 7)
Starting finding coordinates for  gis_osm_landuse_a_free_1  of shape  (529284, 4)
Added coordinates to  gis_osm_landuse_a_free_1 , now shape is  (529284, 6)
Starting finding coordinates for  gis_osm_natural_a_free_1  of shape  (1198, 4)
Added coordinates to  gis_osm_natural_a_free_1 , now shape is  (1198, 6)
Starting finding coordinates for  gis_osm_natural_free_1  of shape  (323206, 4)
Added coordinates to  gis_osm_natural_free_1 , now shape is  (323206, 6)
Starting finding coordinates for  gis_osm_places_a_free_1  of shape  (672, 5)
Added coordinates to  gis_osm_places_a_free_1 , now shape is  (672, 7)
Starting finding coordinates for  gis_osm_places_free_1  of shape  (34338, 5)
Added coordinates to  gis_osm_places_free_1 , now shape is  (34338, 7)
Starting finding coordinates for  gis_osm_pofw_a_free_1  of shape  (11809, 4

In [8]:
# SAVE THE RESULTS IN SEPARATE CSV FILES

os.makedirs('coordinates/', exist_ok=True)

for df_name, df in df_dict.items():

    file_name = 'coordinates_' + df_name[:-4] + '.csv'
    df.to_csv('coordinates/' + file_name)